## ワークスペース設定

In [3]:
import azure.ai.ml 
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azureml.core import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')


try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": ws.subscription_id,
        "resource_group": ws.resource_group,
        "workspace_name": ws.name,
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)


Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/skilling-ml2/code/Users/localdev/github/mslearn-skill2022/.azureml/config.json


In [69]:
import mlflow
from mlflow.tracking.client import MlflowClient
tracking_uri = ml_client.workspaces.get(name=ws.name).mlflow_tracking_uri
mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient()

## modelのローカルテスト

In [70]:
run_id="boring_okra_ys612y5fdv"
client.list_artifacts(run_id)
client.download_artifacts(run_id, "model", ".")

import joblib
loaded_model = joblib.load("model/model.pkl")

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/skilling-ml2/code/Users/localdev/github/mslearn-skill2022/train-evaluate-regression-models/model'

In [20]:
import numpy as np
# Create a numpy array containing a new observation (for example tomorrow's seasonal and weather forecast information)
X_new = np.array([[1,1,0,3,1,1,0.226957,0.22927,0.436957,0.1869]]).astype('float64')
print ('New sample: {}'.format(list(X_new[0])))

# Use the model to predict tomorrow's rentals
result = loaded_model.predict(X_new)
print('Prediction: {:.0f} rentals'.format(np.round(result[0])))

New sample: [1.0, 1.0, 0.0, 3.0, 1.0, 1.0, 0.226957, 0.22927, 0.436957, 0.1869]
Prediction: 129 rentals


In [22]:
# An array of features based on five-day weather forecast
X_new = np.array([[0,1,1,0,0,1,0.344167,0.363625,0.805833,0.160446],
                  [0,1,0,1,0,1,0.363478,0.353739,0.696087,0.248539],
                  [0,1,0,2,0,1,0.196364,0.189405,0.437273,0.248309],
                  [0,1,0,3,0,1,0.2,0.212122,0.590435,0.160296],
                  [0,1,0,4,0,1,0.226957,0.22927,0.436957,0.1869]])

# Use the model to predict rentals
results = loaded_model.predict(X_new)
print('5-day rental predictions:')
for prediction in results:
    print(np.round(prediction))

New sample: [0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.344167, 0.363625, 0.805833, 0.160446]
5-day rental predictions:
549.0
928.0
432.0
397.0
470.0


## オンラインマネージドエンドポイントにデプロイ

In [26]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "bike-share-endpoint-" + str(uuid.uuid4())[:8]

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "sdkv2-daily-bike-share",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

endpoint = ml_client.begin_create_or_update(endpoint)

print(f"Endpint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpint bike-share-endpoint-2206ae4b provisioning state: Succeeded


In [27]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpint "bike-share-endpoint-2206ae4b" with provisioning state "Succeeded" is retrieved


In [ ]:
registered_model_name = "azureml_boring_okra_ys612y5fdv_output_mlflow_log_model"


In [9]:
# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name=registered_model_name, version="1")

In [28]:
# create an online deployment.
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F2s_v2",
    instance_count=1,
)

blue_deployment = ml_client.begin_create_or_update(blue_deployment)

Check: endpoint bike-share-endpoint-2206ae4b exists
Creating/updating online deployment blue Done (5m 52s)


..................................................................

## Requestのテスト

In [13]:
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [23]:
%%writefile {deploy_dir}/sample-request.json
{
  "input_data": [
        [1.0, 1.0, 0.0, 3.0, 1.0, 1.0, 0.226957, 0.22927, 0.436957, 0.1869],
        [0,1,1,0,0,1,0.344167,0.363625,0.805833,0.160446],
        [0,1,0,1,0,1,0.363478,0.353739,0.696087,0.248539],
        [0,1,0,2,0,1,0.196364,0.189405,0.437273,0.248309],
        [0,1,0,3,0,1,0.2,0.212122,0.590435,0.160296],
        [0,1,0,4,0,1,0.226957,0.22927,0.436957,0.1869]
    ]
}

Overwriting ./deploy/sample-request.json


In [24]:
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)

'[129.210589270042, 548.8507783701633, 927.5562696858865, 431.51742301778836, 396.6375139680033, 470.3986407767837]'